<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Sayeh Jarollahi <br> 
   **Student ID**: 98101339 <br> <br>
    

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

<div dir = "rtl">
<font size="4">
    در ابتدا دیتاست مربوط به مسئله را میسازیم. اینکار با استفاده از  sklearn انجام میشود.
    <br>
    پس از آن با استفاده از gradiant descent
    و در ایتریشن های متوالی مقدار بهینه وزن استفاده میشود. 
</div>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score

In [5]:
X, y = make_regression(n_samples=50_000, n_features=3, noise=25)

In [6]:
class ZeroRegularizer:
    
    def regularize(self, w):
        return 0
    
    def get_reg_cost(self, w):
        return 0

In [7]:
class LinearRegression:
    
    def __init__(self, n_feats, regularization):
        self.W = np.zeros(n_feats)
        self.bias = 0
        self.learning_rate = 0.1
        self.iterations = 2000
        self.regularization = regularization


    def fit(self, X, y):
        self.W = np.zeros(X.shape[1])
        self.bias = 0 
        for i in range(self.iterations):
            y_hat = np.dot(X, self.W) +self.bias
            #db = (1/X.shape[0]) * np.sum(y_hat-y) 
            dw = (1/X.shape[0]) * np.dot(X.T, (y_hat-y)) + self.regularization.regularize(self.W)
            self.W = self.W - self.learning_rate * dw
            #self.bias = self.bias - self.learning_rate * db
        
    def predict(self, X):
        return np.dot(X, self.W) + self.bias

In [8]:
def test_model(X, y, regularizer, test_size=0.2, random_state=1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= test_size, random_state= random_state)

    linreg = LinearRegression(X.shape[1], regularizer)
    linreg.fit(X_train, y_train)

    predictions = linreg.predict(X_test)
    print("r2 score", r2_score(y_test, predictions))

In [9]:
test_model(X, y, ZeroRegularizer())

r2 score 0.9645790402026584


<div dir = "rtl">
<font size="4">
    مشابه با آنچه در کلاس تدریس شده، کد مربوط به linear regression
    پیاده سازی شده است. ابتدا داده اصلی را به دو بخش تست و ترین تقسیم میکنیم. 80 درصد از کل دیتا، مربوط به دیتای ترین و بقیه مربوط به دیتای تست خواهد بود. پس از آن با استفاده از فیت کردن داده ترین روی مدل، آن را ترین میکنیم. در هربار iteration
    ابتدا تخمین ورودی را به دست می آوریم و اختلاف آن را با مقدار واقعی در نظر میگیریم. با استفاده از محاسبه گرادیان به کمک MSE
    وزن و بایاس را آپدیت میکنیم. 
    <br>
    در انتها مقدار ارور را بر روی داده تست محاسبه میکنیم.
    توجه: برای آنکه از مدل بالا بتوانیم در بخش های آینده نیز استفاده کنیم، یک عبارت regularization
    به آن اضافه کردیم که در صورتی که نرمال سازی نداشته باشیم مقدار صفر را دارد و در غیر اینصورت overwrite 
    یک کلاس است که در آن با کمک تابع regularize
    مقدار را اضافه میکنیم. 
    
</div>

<div dir = "rtl">
<font size="5">
  <b> 
      نرمال سازی با کمک Lasso
      :
  </b>
    
</div>

Lasso Regression uses L1-regularization norm to prevent the model from overfitting. We define the regularization factor. After that, normal linear regression can be used. 

In [10]:
class LassoRegularizer:
    
    def __init__(self, l):
        self.l = l
    
    def regularize(self, w):
        return self.l * np.sign(w) 
    

In [11]:
test_model(X, y, LassoRegularizer(0.1))

r2 score 0.9645820585791383


As we said before, Lasso Regression prevents overfitting. The MSE loss in this approach is  less than MSE loss in the normal Linear Regression. 

## Ridge Regression 

Ridge Regression uses l2 regularization to prevent model from overfitting. If the model gets complex, the cost function increases. Consequently, the model prevents itself from overfitting. 

In [12]:
class RidgeRegularizer:
    
    def __init__(self, l):
        self.l = l
        
    def regularize(self, w):
        return 2 * self.l * w

In [13]:
test_model(X, y, RidgeRegularizer(0.15))

r2 score 0.9142553412210468


In [14]:
    
class ElasticRegularizer:
    
    def __init__(self, l = 0.1, l_ratio = 0.5):
        self.l = l 
        self.l_ratio = l_ratio

    def regularize(self, w):
        l1_derivation = self.l * self.l_ratio * np.sign(w)
        l2_derivation = self.l * (1 - self.l_ratio) * w
        return (l1_derivation + l2_derivation)
    

In [15]:
test_model(X, y, ElasticRegularizer())

r2 score 0.9624565535452143


As we can see, after adding regularizations the r2 scores improve. 
This means that Lasso, Ridge regressions and elsastic Net prevent the model from overfitting. The do not let the model get complex and fit completely to the training data. Hence, they perform better in the testing phase. 
But when data is not noisy, it is normal for the ridge, lasso and elastic net to perform worse than linear regression. 